In [1]:
import os,sys,time,random,math,time
import tarfile, zipfile

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.cross_validation import train_test_split, StratifiedShuffleSplit
from sklearn.linear_model import LinearRegression,Ridge

from sklearn import decomposition, datasets, ensemble
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer,precision_score, recall_score, f1_score, average_precision_score, accuracy_score, mean_absolute_error

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

import matplotlib.pyplot as plt
from IPython.display import display, Image

import xgboost as xgb


from subprocess import check_output
datadir="./input/"
print(check_output(["ls", datadir]).decode("utf8"))

%matplotlib inline  


test.csv.zip
train.csv.zip



In [2]:
def loadData(datadir,filename):
    # Load the wholesale customers dataset
    #data = pd.read_csv(filename)
    data = ''
    print ("loading: "+datadir+filename)
    try:
        if zipfile.is_zipfile(datadir+filename):
            z = zipfile.ZipFile(datadir+filename)
            filename = z.open(filename[:-4])
        else:
            filename=datadir+filename
        data = pd.read_csv(filename, parse_dates=True)  
        print ("Dataset has {} samples with {} features each.".format(*data.shape))
    except Exception as e:
        print ("Dataset could not be loaded. Is the dataset missing?")
        print(e)
    return data

def writeData(data,filename):
    # Load the wholesale customers dataset
    try:
        data.to_csv(filename, index=False)
    except Exception as e:
        print ("Dataset could not be written.")
        print(e)
    verify=[]
    try:
        with open(filename, 'r') as f:
            for line in f:
                verify.append(line)
        f.closed
        return verify[:5]
    except IOError:
        sys.std
        
def LabelEncoder(data):
    # lifted in parts from:
    #https://www.kaggle.com/mmueller/allstate-claims-severity/yet-another-xgb-starter/code
    features = data.columns
    cats = [feat for feat in features if 'cat' in feat]
    for feat in cats:
        data[feat] = pd.factorize(data[feat], sort=True)[0]
    return data

# XGB!

def xgbfit(X_train,y_train):
    dtrain = xgb.DMatrix(X_train, label=y_train)
    

    xgb_params = {
        'seed': 0,
        'colsample_bytree': 0.7,
        'silent': 1,
        'subsample': 0.7,
        'learning_rate': 0.075,
        'objective': 'reg:linear',
        'max_depth': 6,
        'num_parallel_tree': 1,
        'min_child_weight': 1,
        'eval_metric': 'mae',
    }

    start_time = time.time()
    res = xgb.cv(xgb_params, dtrain, num_boost_round=750, nfold=4, seed=42, stratified=False,
                 early_stopping_rounds=15, verbose_eval=100, show_stdv=True, maximize=False)
    print("fit time:{}s".format(round((time.time()-start_time), 3) ))

    best_nrounds = res.shape[0] - 1
    cv_mean = res.iloc[-1, 0]
    cv_std = res.iloc[-1, 1]
    print('CV-Mean: {0}+{1}'.format(cv_mean, cv_std))
    # XGB Train!
    start_time = time.time()
    gbdt = xgb.train(xgb_params, dtrain, best_nrounds)
    print("Train time:{}s".format(round((time.time()-start_time), 3) ))
    return gbdt

In [3]:
data = loadData(datadir,'train.csv.zip')
display(data.info())
display(data.head(5))

test_data= loadData(datadir,'test.csv.zip') 
display(test_data.info())
display(test_data.head(5))

loading: ./input/train.csv.zip
Dataset has 188318 samples with 132 features each.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188318 entries, 0 to 188317
Columns: 132 entries, id to loss
dtypes: float64(15), int64(1), object(116)
memory usage: 189.7+ MB


None

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


loading: ./input/test.csv.zip
Dataset has 125546 samples with 131 features each.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125546 entries, 0 to 125545
Columns: 131 entries, id to cont14
dtypes: float64(14), int64(1), object(116)
memory usage: 125.5+ MB


None

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,A,B,A,A,A,A,A,A,B,...,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,A,B,A,B,A,A,A,A,B,...,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,A,B,A,B,B,A,B,A,B,...,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,A,A,A,A,B,A,A,A,A,...,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,B,A,A,A,A,B,A,A,A,...,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


###  Pre Proccessing

In [4]:
# combine the two frames so we can encode the labels!
test_data['loss']=0

lengthofData=len(data)
lengthoftest_data=len(test_data)

print("data:",lengthofData)
print("test:",lengthoftest_data)

combineddata=pd.concat([data,test_data])
lengthofcombined=len(combineddata)
print("combined:",lengthofcombined)

# the categorical data that we need in a number format
combineddata=LabelEncoder(combineddata)

# time to split the data back apart!
data=combineddata.iloc[:lengthofData].copy()
test_data=combineddata.iloc[lengthofData:].copy()
test_data.drop(['loss'],1,inplace=True) # didn't have this column before, make it go away!


x_test = test_data.copy()
x_test.drop(['id'],1,inplace=True)

# we don't want the ID columns in X, and of course not loss either
x=data.drop(['id','loss'],1)
# loss is our label
y=data['loss']

#minmax scaler
scaler= MinMaxScaler() 
x = scaler.fit_transform(x)
x_test_data = scaler.fit_transform(x_test)

#display(x[:5])
#display(y.head(5))

print("Pre-Processing done")
print("data:",len(x))
print("labels:",len(y))
print("test:",len(x_test_data))


('data:', 188318)
('test:', 125546)
('combined:', 313864)
Pre-Processing done
('data:', 188318)
('labels:', 188318)
('test:', 125546)


### Stacking, Layer 1

In [46]:
# OK let's actually do some ML
regrList=[] # a list of regressions to use
#regrList.append(LinearRegression())
regrList.append(ExtraTreesRegressor())
regrList.append(Ridge())
    
regrList.append(RandomForestRegressor(n_estimators=10,
                                      #criterion = 'mae',
                                      n_jobs =-1, 
                                      random_state=42))

Split the data into k-folds(divisions). train the regressors on each combination of k-1 folds, and then predict on the held-out fold. Preserve the prediction of each regressor for the next layer.

In [47]:
#prepare the fold divisions

data_size=x.shape[0]
print "size of train data:",data_size
folds=[]
num_folds=5
fold_start=0
for k in range(num_folds-1):
    fold_end=((data_size/num_folds)*(k+1))
    folds.append((fold_start,fold_end))
    fold_start=fold_end
folds.append((fold_start,data_size))
print "folds at:",folds
print "fold size:", (data_size/num_folds)
print "train size:",(data_size/num_folds)*(num_folds-1)

count=0
for i in folds:
    count+=i[1]-i[0]
print count

size of train data: 188318
folds at: [(0, 37663), (37663, 75326), (75326, 112989), (112989, 150652), (150652, 188318)]
fold size: 37663
train size: 150652
188318


In [ ]:
x_layer2=[]
start_time1 = time.time()

for fold_start,fold_end in folds:
    start_time1 = time.time()
    fold_result=[]
    
    X_test = x[fold_start:fold_end].copy()
    y_test = y[fold_start:fold_end].copy()
    X_train=np.concatenate((x[:fold_start], x[fold_end:]), axis=0)
    y_train=np.concatenate((y[:fold_start], y[fold_end:]), axis=0)
    print "\nfolding! len test {}, len train {}".format(len(X_test),len(X_train))
    
    for i in range(len(regrList)): # for each of the regressions we use, fit/predict the data
        start_time = time.time()
        regrList[i].fit(X_train,y_train)
        print("\nfit time:{}s".format(round((time.time()-start_time), 3) ))

        start_time = time.time()
        print(regrList[i])
        curr_predict=regrList[i].predict(X_test)
        if fold_result == []:
            fold_result = np.array(curr_predict.copy())
        else:
            fold_result = np.column_stack((fold_result,curr_predict))
        
        print("predict time:{}s".format(round((time.time()-start_time), 3) ))
        #show some stats on that last regressions run    
        print("Mean abs error: {:.2f}".format(np.mean(abs(curr_predict - y_test))))
        print("Score: {:.2f}".format(regrList[i].score(X_test, y_test)))
    
    #XGB -- it doesn't fit the pattern of scikit, so do it seperatly
    dtest = xgb.DMatrix(X_test)
    gbdt=xgbfit(X_train,y_train)

    # now do a prediction and spit out a score(MAE) that means something
    start_time = time.time()
    curr_predict=gbdt.predict(dtest)
    fold_result = np.column_stack((fold_result,curr_predict))  
    print("XGB Mean abs error: {:.2f}".format(np.mean(abs(curr_predict - y_test))))
    print("XGB predict time:{}s".format(round((time.time()-start_time), 3) ))
    
    if x_layer2 == []:
        x_layer2=fold_result
    else:
        x_layer2=np.append(x_layer2,fold_result,axis=0)
        
    print "--layer2 length:",len(x_layer2)
    print "--layer2 shape:",np.shape(x_layer2)
    print("Fold run time:{}s".format(round((time.time()-start_time1), 3) ))   
print("Full run time:{}s".format(round((time.time()-start_time0), 3) ))   



folding! len test 37663, len train 150655

fit time:61.776s
ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
          verbose=0, warm_start=False)
predict time:0.492s
Mean abs error: 1327.48
Score: 0.46

fit time:3.542s
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
predict time:0.011s
Mean abs error: 1335.61
Score: 0.49


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:22: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.



fit time:33.46s
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=42,
           verbose=0, warm_start=False)
predict time:0.325s
Mean abs error: 1328.68
Score: 0.46
[0]	train-mae:2810.38+4.10425	test-mae:2810.39+13.8376
[100]	train-mae:1160.85+1.873	test-mae:1205.24+9.52449
[200]	train-mae:1120.23+2.3826	test-mae:1190.7+9.58269



### train layer 2

In [54]:
print len(x_layer2)
print len(y)

#  train/validation split
X_layer2_train, X_layer2_validation, y_layer2_train, y_layer2_validation = train_test_split( x_layer2,
                                                                                y,
                                                                                test_size=0.25,
                                                                                random_state=42)
layer2_regr=LinearRegression()

layer2_regr.fit(X_layer2_train,y_layer2_train)

layer2_predict=layer2_regr.predict(X_layer2_validation)

#show some stats on that last regressions run    
print("Mean abs error: {:.2f}".format(np.mean(abs(layer2_predict - y_layer2_validation))))
print("Score: {:.2f}".format(layer2_regr.score(X_layer2_validation, y_layer2_validation)))


#with LinearReg: Mean abs error: 1238.52

188318
188318
Mean abs error: 1238.64
Score: 0.52


In [55]:
# The XGB version of layer 2
print len(x_layer2)
print len(y)

#  train/validation split
X_layer2_train, X_layer2_validation, y_layer2_train, y_layer2_validation = train_test_split( x_layer2,
                                                                                y,
                                                                                test_size=0.25,
                                                                                random_state=42)
#XGB -- it doesn't fit the pattern of scikit, so do it seperatly
dtest = xgb.DMatrix(X_layer2_validation)
layer2_gbdt=xgbfit(X_layer2_train,y_layer2_train)

# now do a prediction and spit out a score(MAE) that means something
start_time = time.time()
print("XGB Mean abs error: {:.2f}".format(np.mean(abs(layer2_gbdt.predict(dtest) - y_layer2_validation))))
print("XGB predict time:{}s".format(round((time.time()-start_time), 3) ))
#with LinearReg: XGB Mean abs error: 1205.77

188318
188318
[0]	train-mae:2810.91+3.90114	test-mae:2810.93+12.0785
fit time:6.48s
CV-Mean: 1207.10253925+5.35427884679
Train time:1.24s
XGB Mean abs error: 1206.63
XGB predict time:0.029s


### Predict layer 1 on test 

In [56]:
x_layer2_test = []
for i in range(len(regrList)): # for each of the regressions we use, fit/predict the data
    start_time = time.time()
    print(regrList[i])
    curr_predict=regrList[i].predict(x_test_data)
    
    print "---before x_layer2 shape:",np.shape(x_layer2),"x_layer2 len",len(x_layer2)

    print("predict time:{}s".format(round((time.time()-start_time), 3) ))

    if x_layer2_test == []:
        print "x_layer2 empty"
        x_layer2_test = np.array(curr_predict.copy())
    else:
        print "Fold_result appended"
        x_layer2_test = np.column_stack((x_layer2_test,curr_predict))

    print "---x_layer2 shape:",np.shape(x_layer2_test),"x_layer2 len",len(x_layer2_test)
    print curr_predict


#XGB -- it doesn't fit the pattern of scikit, so do it seperatly
dtest = xgb.DMatrix(X_test)

# now do a prediction and spit out a score(MAE) that means something
#start_time = time.time()
curr_predict=gbdt.predict(dtest)
x_layer2_test = np.column_stack((x_layer2_test,curr_predict))
#print("Mean abs error: {:.2f}".format(np.mean(abs(cache[i+1] - y_test))))
#print("predict time:{}s".format(round((time.time()-start_time), 3) ))

print "--layer2 length:",len(x_layer2_test)
print "--layer2 shape:",np.shape(x_layer2_test)
print("Fold run time:{}s".format(round((time.time()-start_time1), 3) ))   

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
          verbose=0, warm_start=False)
---before x_layer2 shape: (188318, 3) x_layer2 len 188318
predict time:2.523s
x_layer2 empty
---x_layer2 shape: (125546,) x_layer2 len 125546
[  2378.327   2754.163  10441.615 ...,   3100.405   1150.739   2927.729]
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
---before x_layer2 shape: (188318, 3) x_layer2 len 188318
predict time:0.096s
Fold_result appended
---x_layer2 shape: (125546, 2) x_layer2 len 125546
[  1155.44633825   1986.3797948   11490.70006385 ...,   2823.69355514
   1076.85379674   4646.9250013 ]
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:11: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


---before x_layer2 shape: (188318, 3) x_layer2 len 188318
predict time:0.869s
Fold_result appended
---x_layer2 shape: (125546, 3) x_layer2 len 125546
[ 2205.797  2107.822  8917.102 ...,  3224.901  1168.459  4035.305]
--layer2 length: 125546
--layer2 shape: (125546, 3)
Fold run time:308.009s


In [57]:
display("size of original test data:",len(x_test_data))
display("Test shape:",np.shape(x_layer2_test))
display("train shape:",np.shape(x_layer2))

print("sample of layer2 test:\n",x_layer2_test[:4])

print("x_layer2_test mean:",x_layer2_test.mean( axis=0))
print("x_layer2 mean:",x_layer2.mean(axis=0))
train_layer2_col0_mean=x_layer2.mean(axis=0)[0]

print("x_layer2_test std:",x_layer2_test.std( axis=0)) 
print("x_layer2 std:",x_layer2.std(axis=0))

# notice that column 0 has a significantly higher mean and std
# here's a hack to not fix that for now!

# check which row in column 0 are significantly far from the mean
problem_column=x_layer2_test.T[0]
outliers=[]
for i in range(len(problem_column)):
    if problem_column[i]>30000:
        outliers.append((i,problem_column[i]))
print("num outliers:",len(outliers))

#for each problem child, set them to the average value from the train set, to null the affect some
for o in outliers:
    problem_column[o[0]]=train_layer2_col0_mean
    
print(problem_column[o[0]])

#check outliers again
problem_column=x_layer2_test.T[0]
outliers=[]
for i in range(len(problem_column)):
    if problem_column[i]>30000:
        outliers.append((i,problem_column[i]))
print("num outliers:",len(outliers))

print(x_layer2_test.T[0][o[0]]) # verify that the change made it all the way to the original

'size of original test data:'

125546

'Test shape:'

(125546, 3)

'train shape:'

(188318, 3)

('sample of layer2 test:\n', array([[  2378.327     ,   1155.44633825,   2205.797     ],
       [  2754.163     ,   1986.3797948 ,   2107.822     ],
       [ 10441.615     ,  11490.70006385,   8917.102     ],
       [  7953.318     ,   5737.1042642 ,   4967.937     ]]))
('x_layer2_test mean:', array([ 3103.57747562,  3056.02085022,  3087.85538827]))
('x_layer2 mean:', array([ 3093.08979795,  3037.45612893,  3093.94178225]))
('x_layer2_test std:', array([ 2237.08236188,  2016.1297939 ,  2242.43405966]))
('x_layer2 std:', array([ 2273.38105749,  2017.89115492,  2266.12894136]))
('num outliers:', 6)
3093.08979795
('num outliers:', 0)
3093.08979795


### Predict Layer 2

In [58]:
test_data['loss']=layer2_regr.predict(x_layer2_test)

result=test_data[['id','loss',]]
output_fname="result_submission_stack.csv"
display(writeData(result,output_fname))



['id,loss\n',
 '4,1813.93600817\n',
 '6,2195.0498681\n',
 '9,10455.2630284\n',
 '12,6177.72526016\n']

In [59]:
#the XGB version:

dtest = xgb.DMatrix(x_layer2_test)
test_data['loss']=layer2_gbdt.predict(dtest)

result=test_data[['id','loss',]]
output_fname="result_submission_stack_xgb.csv"
display(writeData(result,output_fname))



['id,loss\n',
 '4,1685.57458496\n',
 '6,1883.73486328\n',
 '9,8873.38964844\n',
 '12,5509.67236328\n']

In [60]:
#let's have a look at the std of the result, as a cross check
print("result std:",result.std(axis=0))

('result std:', id      170098.328125
loss      1851.717773
dtype: float32)


# EOF 